In [1]:
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained('./models/.')
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('./models/tf_tokenizer/.')
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model = model, tokenizer = tokenizer, return_all_scores=True)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at ./models/..
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [2]:
import pandas as pd

In [12]:
df = pd.read_excel('tweets_labeled.xlsx')
l = 'Label (critical: 1/neutral: 0)'
f = 'full_text'

In [11]:
print(df.head())
print(df[l].iloc[599])
print(df.size)

   Unnamed: 0            id created_at  \
0         0.0  1.500745e+18 2022-03-07   
1         1.0  1.500745e+18 2022-03-07   
2         2.0  1.500745e+18 2022-03-07   
3         3.0  1.500745e+18 2022-03-07   
4         4.0  1.500745e+18 2022-03-07   

                                           full_text  retweet_count  \
0  RT @MeenaDasNarayan: You are a scamster, who h...            4.0   
1  RT @EricWishart: Because of #COVID, the mask s...           74.0   
2  RT @KentPage: Amidst deadly conflict and #COVI...           14.0   
3  RT @phyz_Buni: The suppression of #Ivermectin ...            1.0   
4  Total Covid Death Distribution by County  For ...            0.0   

   Label (critical: 1/neutral: 0)  
0                             1.0  
1                             1.0  
2                             0.0  
3                             0.0  
4                             0.0  
1.0
15690


In [20]:
r = pipe("i hate covid")[0]
print(r)
result = {}
for element in r:
    result[element['label']] = element['score']
label = max(result, key=result.get)
print(label)
#print(result)

[{'label': 'LABEL_0', 'score': 0.39870285987854004}, {'label': 'LABEL_1', 'score': 0.60129714012146}]
LABEL_1


In [21]:
def find_label(tweet):
    r = pipe(tweet)[0]
    result = {}
    for element in r:
        result[element['label']] = element['score']
    label = max(result, key=result.get)
    return label

In [22]:
n = df.size
for i in range(600, n):
    # get the label
    label = find_label(df[f].iloc[i])
    if label == "LABEL_0":
        # neutral
        df[l].iloc[i] = 0
    elif label == "LABEL_1":
        # critical
        df[l].iloc[i] = 1
    else:
        # error
        raise ValueError(f"the label is random at index {i}")


C:\Users\risto\AppData\Local\Temp\ipykernel_19800\3470471199.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[l].iloc[i] = 0
C:\Users\risto\AppData\Local\Temp\ipykernel_19800\3470471199.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[l].iloc[i] = 1


IndexError: single positional indexer is out-of-bounds

In [31]:
df.to_excel('tweets_labeled_all.xlsx')

In [33]:
print(df[l].iloc[2615])

IndexError: single positional indexer is out-of-bounds